### CMPE493 (Introduction to Information Retrieval) | Assignment 3: Cutting It Short
### 2014400072

## Models

**Model-1** constructs the vector representation of a sentence by taking the simple average of the vectors of all the words in that sentence.

**Model-2** constructs the vector representation of a sentence by multiplying the vector of a word with that word's **tf-idf score** for all the words in that sentence and then taking the simple average of the results.

Both models use the same tokenization mechanism for extracting words from a sentence. Tokenization includes removing some punctuation marks and lower-casing.

After the sentences of an article are vectorized, both models run **k-means** algorithm with $k = \sqrt{<length-of-article>}$ where `<length-of-article>` is the number of sentences in that article. Clusters are constructed with respect to **cosine similarity** and the most prominent sentence of each cluster is concatenated to form a summary.

## Required Libraries

In [1]:
import numpy as np
import os
import pickle
import re
import rouge

## Some Parameters and Global Variables

In [2]:
VECTOR_PATH = '../glove.6B.200d.pkl'
ARTICLES_PATH = '../articles/'
GOLD_SUMMARIES_PATH = '../gold_summaries/'

VECTOR_DIM = 200  # Dimension of GloVe vectors
FOLD_K = 10  # K-Fold Cross-Validation parameter

# Regex pattern used to replace some punctuation marks in article texts with space.
PUNCTUATIONS_RGX = r'(\"|\!|\^|\%|\<|\+|\~|\*|\;|\:|\(|\?|\&|\}|\]|\||\,|\'|\)|\-|\#|\`|\@|\/|\$|\_|\{|\.|\>|\[|\\|\=)'

# Regex pattern used to extract sentences from a paragraph.
SENTENCE_SEPARATOR_RGX = r'''(?:(?<=[a-zA-Z0-9][.!?:])|(?<=[.!?:]["']))\s+(?=[a-zA-Z0-9"'])'''


ARTICLE_SENTENCES_DICT = {}  # Maps articles (filenames) to a List of sentences of that article.
ARTICLE_GOLD_SUMMARY_DICT = {}  # Maps articles (filenames) to a String of gold summary of that article.
ARTICLE_TF_DICT = {}  # Maps articles (filenames) to a Dictionary which maps terms in that article to their frequency.

ROUGE = rouge.Rouge()

with open(VECTOR_PATH, 'rb') as f:
    GLOVE_DICT = pickle.load(f)

## Functions

In [3]:
def article_by_sentences(article_text):
    """
    Returns a List of sentences for given `article` (str). Article is assumed to contain one paragraph at a line.
    """
    paragraphs = [line.strip() for line in re.split(r'(?:\r?\n)+', article_text) if len(line.strip()) != 0]

    # Maps each paragraph to a List of sentences of that paragraph.
    paragraph_sentences = [re.split(SENTENCE_SEPARATOR_RGX, paragraph) for paragraph in paragraphs]

    return [sentence for sentence_list in paragraph_sentences
            for sentence in sentence_list if re.search(r'[a-zA-Z0-9]', sentence)]

In [4]:
def tokenize(sentence):
    """
    Returns a List of tokens (words) for given `sentence`. Tokenization includes removing some punctuation marks and
    lower-casing.
    """
    return [token.lower() for token in re.sub(PUNCTUATIONS_RGX, " ", sentence).split()]

In [5]:
def sentence_to_vector(sentence, word_weight_dict=None):
    """
    Returns the vector representation of the given `sentence`. The vector is calculated as the average
    of the vectors of the words in `sentence`.

    `word_weight_dict` is a Dictionary object which maps terms in `sentence` to a number (weight). If not None,
    the vectors of the words are multiplied by the corresponding weights while calculating the average.
    """
    # Returns a List of zeros (length VECTOR_DIM) if none of the words in `sentence` exists in GLOVE_DICT.
    return [round(x, 5) for x in np.mean(
        [np.array(GLOVE_DICT[word]) * (word_weight_dict.get(word, 0) if word_weight_dict else 1)
         for word in tokenize(sentence) if word in GLOVE_DICT] or [np.zeros(VECTOR_DIM)], axis=0)]

In [6]:
def length_of_vector(vector):
    """
    Returns the length (magnitude) of given `vector`.
    """
    return round(np.sqrt(sum([elem**2 for elem in vector])), 5)

In [7]:
def cosine_similarity(vec1, vec2):
    """
    Returns the cosine similarity of the given vectors `vec1` and `vec2`.
    """
    dot_product = sum([e1*e2 for (e1, e2) in zip(vec1, vec2)])
    return 0.0 if dot_product == 0 else round(dot_product/(length_of_vector(vec1) * length_of_vector(vec2)), 5)

In [8]:
def k_means(k, data_list):
    """
    Separates given `data_list` into `k` clusters and returns a Tuple of (<cluster-centroids>, <data-clusters>).

    Clusters are formed according to the cosine similarity and they are labeled as 0, 1, ..., k-1.
    Initial centroids are selected as the random elements of `data_list` and at most 30 iterations are applied.

    `k` is the number of clusters (int).
    `data_list` is a List of instances; the instances can be scalar numbers or vectors (Lists of numbers).

    <cluster-centroids> is a List of length `k`. ith element is the centroid of cluster-i.
    <data-clusters> is a List, same length as `data_list`. ith element is the cluster (label) of the ith element
    of `data_list`.
    """

    # Random data instances are chosen as the initial cluster centroids. ith element is the centroid of cluster-i.
    unique_data_list = list(set(tuple(x) for x in data_list))
    cluster_centroids = [unique_data_list[i] for i in np.random.choice(range(len(unique_data_list)), k, replace=False)]

    # Stores the clusters to which the data instances are currently assigned.
    data_clusters = []  # ith element is the cluster of the ith data instance in `data_list`.

    step = 0  # Current iteration step.

    while True:
        step += 1

        # Maps clusters to a List of data instances assigned to that cluster.
        cluster_members_dict = {i: [] for i in range(k)}

        # Stores the clusters to which the data instances are assigned after the re-assignment phase.
        # ith element is the cluster of the ith data instance in `data_list`.
        new_data_clusters = [-1 for _ in range(len(data_list))]

        # Re-assignment of the data instances to the clusters.
        for (i, data) in enumerate(data_list):
            similarity_cluster_tuples = []  # List of (<cosine-similarity>, <cluster>) pairs

            for (cluster, centroid) in enumerate(cluster_centroids):
                similarity_cluster_tuples.append((cosine_similarity(data, centroid), cluster))

            # Sorts by cosine similarity in descending order and takes the cluster with maximum cosine similarity.
            chosen_cluster = sorted(similarity_cluster_tuples, reverse=True)[0][1]
            cluster_members_dict[chosen_cluster].append(data)  # Data instance is assigned to the most similar centroid.
            new_data_clusters[i] = chosen_cluster

        if np.array_equal(new_data_clusters, data_clusters) or (step == 30):
            break  # Stops if none of the data instances is re-assigned to a different cluster.

        data_clusters = new_data_clusters

        # Updates the centroids.
        for cluster in cluster_members_dict:
            if len(cluster_members_dict[cluster]) != 0:  # If some instance is assigned to this cluster, updates it.
                cluster_centroids[cluster] = [round(x, 5) for x in np.mean(cluster_members_dict[cluster], axis=0)]

    return cluster_centroids, data_clusters

In [9]:
def form_summary(sentences, vectors, centroids, clusters):
    """
    Forms and returns a summary for an article given as `sentences`. The most prominent sentence of each cluster is
    taken and concatenated to form a summary.

    :param sentences: A List of sentences of an article.
    :param vectors: A List of vectors for sentences in `sentences`.
    :param centroids: A List of centroid vectors. ith element is the centroid for cluster-i.
    :param clusters: A List of cluster labels (int). ith element is the cluster of the ith sentence in `sentences`.
    :return: Summary (str).
    """

    # Maps clusters to a List of (<cosine-similarity>, <sentence-index>) pairs for sentences that belong that cluster.
    # <cosine-similarity> is measured between a sentence and the centroid of the cluster that sentence belongs to.
    cluster_sentences_dict = {i: [] for i in range(len(centroids))}

    for (i, vector) in enumerate(vectors):
        cluster = clusters[i]  # Cluster of the ith sentence/vector.
        cluster_sentences_dict[cluster].append((cosine_similarity(centroids[cluster], vector), i))

    summary = ''

    for similarity_index_tuples in cluster_sentences_dict.values():
        if similarity_index_tuples:  # If not empty List
            # Sorts by cos-similarity in descending order and takes the sentence with the highest similarity.
            summary += sentences[sorted(similarity_index_tuples, reverse=True)[0][1]]

    return summary

In [10]:
def model1_summary(sentences):
    """
    Constructs a summary for given article `sentences` via Model-1 and returns it. `sentences` is a List of sentences.
    """
    vectors = [sentence_to_vector(s) for s in sentences]

    # k_means() returns (<cluster-centroids>, <data-clusters>).
    return form_summary(sentences, vectors, *k_means(int(round(np.sqrt(len(vectors)))), vectors))

In [11]:
def model2_summary(sentences, word_weight_dict):
    """
    Constructs a summary for given article `sentences` via Model-2 and returns it. `sentences` is a List of sentences.
    """
    vectors = [sentence_to_vector(s, word_weight_dict=word_weight_dict) for s in sentences]

    # k_means() returns (<cluster-centroids>, <data-clusters>).
    return form_summary(sentences, vectors, *k_means(int(round(np.sqrt(len(vectors)))), vectors))

In [12]:
def rouge_scores(train_articles, test_articles):
    """
    A helper function for calculating Rouge scores of Model-1 and Model-2 on validation/test data set.

    `train_articles` is a List of train articles (file names).
    `test_articles` is a List of test articles (file names).

    Returns a Tuple of (`model1_scores`, `model2_scores`)
    """
    model1_scores = {'rouge-1': [], 'rouge-2': [], 'rouge-l': []}
    model2_scores = {'rouge-1': [], 'rouge-2': [], 'rouge-l': []}

    # TRAIN MODEL-2

    term_df_dict = {}

    for train_article in train_articles:
        for term in ARTICLE_TF_DICT[train_article]:
            if term not in term_df_dict:  # If `term` is encountered for the first time in this collection...
                term_df_dict[term] = 1  # ... assigns a document-frequency of 1 to it.
            else:
                term_df_dict[term] += 1  # Else, increases its document-frequency by 1.

    for test_article in test_articles:
        # Maps terms (words) to their tf-idf values
        word_weight_dict = {term: tf_idf_score(frequency, term_df_dict.get(term, 1), len(train_articles))
                            for (term, frequency) in ARTICLE_TF_DICT[test_article].items()}

        gold_summary = ARTICLE_GOLD_SUMMARY_DICT[test_article]

        m1_summary = model1_summary(ARTICLE_SENTENCES_DICT[test_article])
        m1_scores = ROUGE.get_scores(m1_summary, gold_summary)

        for (rouge_type, scores) in m1_scores[0].items():
            model1_scores[rouge_type].append(scores['f'])

        m2_summary = model2_summary(ARTICLE_SENTENCES_DICT[test_article], word_weight_dict)
        m2_scores = ROUGE.get_scores(m2_summary, gold_summary)

        for (rouge_type, scores) in m2_scores[0].items():
            model2_scores[rouge_type].append(scores['f'])

    return model1_scores, model2_scores

The model used to calculate TF-IDF score (`w`) of a term t in a document d is as follows:

![tf-idf-weight.png](attachment:tf-idf-weight.png)

The meanings of `tf`, `df` and `N` are explained in the function docstring below:

In [13]:
def tf_idf_score(tf, df, N):
    """
    Calculates tf-idf score of a term t in a document d.
    :param tf: Frequency of t in d (int, 0 <= tf)
    :param df: Document frequency of t (int, 0 < df <= N)
    :param N: Number of documents (int)
    """
    return 0.0 if tf == 0 else round((1 + np.log10(tf)) * np.log10(N/df), 5)

## Program

In [14]:
np.random.seed(256)  # Specifies a random seed to obtain the same results.

The block below reads all the article and gold summary files and constructs `ARTICLE_GOLD_SUMMARY_DICT`, `ARTICLE_TF_DICT` and `ARTICLE_SENTENCES_DICT` dictionary objects.

This block takes about 50sec.

In [15]:
articles = os.listdir(ARTICLES_PATH)

print("Processing article files... %00.00", end='')

for (i, article) in enumerate(articles):
    if (i+1) % 25 == 0:
        for _ in range(5):
            print("\b", end='', flush=True)
        print(f"{100*(i+1)/len(articles):05.2f}", end='')
    
    with open(os.path.join(GOLD_SUMMARIES_PATH, article), errors='ignore') as f:  # Ignores encoding errors.
        ARTICLE_GOLD_SUMMARY_DICT[article] = f.read()

    with open(os.path.join(ARTICLES_PATH, article), errors='ignore') as f:  # Ignores encoding errors.
        article_sentences = article_by_sentences(f.read())

    ARTICLE_TF_DICT[article] = {}

    for sentence in article_sentences:
        for token in tokenize(sentence):
            if token not in ARTICLE_TF_DICT[article]:  # If `token` is encountered for the first time in this article...
                ARTICLE_TF_DICT[article][token] = 1  # ... assigns a term-frequency of 1 to it.
            else:
                ARTICLE_TF_DICT[article][token] += 1  # Else, increases its term-frequency by 1.

    ARTICLE_SENTENCES_DICT[article] = article_sentences

print("\nDONE.")

Processing article files... %100.00
DONE.


In [16]:
# Separates articles randomly into TRAIN and TEST test with ratio 4:1.
np.random.shuffle(articles)
TRAIN_ARTICLES = articles[:int(len(articles)*0.8)]
TEST_ARTICLES = articles[int(len(articles)*0.8):]


print(f"TRAIN DATA SIZE = {len(TRAIN_ARTICLES)}\t\tTEST DATA SIZE = {len(TEST_ARTICLES)}")

TRAIN DATA SIZE = 1780		TEST DATA SIZE = 445


## K-Fold Cross-Validation

This block takes about 6min30sec.

In [17]:
print("Validation is in progress... %00.00", end='')

m1_valid_scores = {'rouge-1': [], 'rouge-2': [], 'rouge-l': []}  # Model-1 validation scores
m2_valid_scores = {'rouge-1': [], 'rouge-2': [], 'rouge-l': []}  # Model-2 validation scores

fold_number = int(np.ceil(len(TRAIN_ARTICLES) / FOLD_K))

for i in range(fold_number):
    for _ in range(5):
        print("\b", end='', flush=True)
    print(f"{100*(i+1)/fold_number:05.2f}", end='')
    
    lower_bound = i * FOLD_K
    upper_bound = lower_bound + FOLD_K

    valid_articles = TRAIN_ARTICLES[lower_bound:upper_bound]  # Validation articles
    other_articles = TRAIN_ARTICLES[:lower_bound] + TRAIN_ARTICLES[upper_bound:]

    m1_scores, m2_scores = rouge_scores(other_articles, valid_articles)

    for (rouge_type, score_list) in m1_scores.items():
        m1_valid_scores[rouge_type].append(np.mean(score_list))

    for (rouge_type, score_list) in m2_scores.items():
        m2_valid_scores[rouge_type].append(np.mean(score_list))

print("\nDONE.")

Validation is in progress... %100.00
DONE.


## Testing

This block takes about 1min30sec.

In [18]:
print("Testing is in progress... ", end='')
m1_test_scores, m2_test_scores = rouge_scores(TRAIN_ARTICLES, TEST_ARTICLES)
print("DONE.")

Testing is in progress... DONE.


## Results

In [19]:
print("\t\tMETRIC\t\tVALIDATION SCORE (mean +- std)\t\tTEST SCORE")

for (model, valid_scores, test_scores) in (('MODEL-1', m1_valid_scores, m1_test_scores), ('MODEL-2', m2_valid_scores, m2_test_scores)):
    print(model)
    for metric in ('rouge-1', 'rouge-2', 'rouge-l'):
        print(f"\t\t{metric}"
              f"\t\t\t{np.mean(valid_scores[metric]):.5f} +- {np.std(valid_scores[metric]):.5f}"
              f"\t\t{np.mean(test_scores[metric]):.5f}")
    print()

		METRIC		VALIDATION SCORE (mean +- std)		TEST SCORE
MODEL-1
		rouge-1			0.49502 +- 0.04185		0.48264
		rouge-2			0.36418 +- 0.04979		0.34789
		rouge-l			0.43774 +- 0.04422		0.42616

MODEL-2
		rouge-1			0.48537 +- 0.04704		0.48616
		rouge-2			0.35295 +- 0.05529		0.35576
		rouge-l			0.42286 +- 0.04956		0.42512



## Conclusion

The results (Rouge scores) above suggest that **Model-1** and **Model-2** perform very similarly. I'm actually surprised by this outcome. Model-1 takes simple average of word vectors to calculate a sentence's vector whereas Model-2 weights the word vectors by their **tf-idf scores** before taking an average. I expect Model-2 to perform better than Model-1 since it is a more complex and it takes into consideration the "values" (*tf-idf scores*) of the words.